In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from copy import deepcopy
import re
import pickle
# Set up Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # run in headless mode
chrome_options.add_argument("--no-sandbox")  # bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
chrome_options.add_argument("--disable-gpu")  # applicable to windows os only
chrome_options.add_argument("start-maximized")  # open Browser in maximized mode
chrome_options.add_argument("disable-infobars")  # disabling infobars
chrome_options.add_argument("--disable-extensions")  # disabling extensions


def scraperLoop(key, text_list, driver, numberOfObjects, special_word = '', time_of_waiting = 4):
    
    time.sleep(1)
    chat_xpath = '/html/body/div/div[1]/div/div[2]/div/form/div/div/textarea'
    chat_input = driver.find_element(By.XPATH, chat_xpath)
    chat_input.send_keys(f'{numberOfObjects}個{key}的同義詞？ {special_word} 用空格分開')
    chat_input.send_keys(Keys.RETURN)
    time.sleep(time_of_waiting)
    for i in range(20):
        try: 
            fetch_xpath = '/html/body/div/div[1]/div/div[1]/div/div[2]/div[1]'
            fetch_output = driver.find_element(By.XPATH, fetch_xpath)
            output = re.sub(r"[（.*?）0-9a-zA-Z]", '', fetch_output.text)
            text_list[key] = re.split(r"[.\s,，、。]+", output)
            break
        except Exception as e:
            time.sleep(1)
    new_chat_xpath = '/html/body/div/div[1]/nav[3]/div[1]/a[2]'
    new_chat_entry = driver.find_element(By.XPATH, new_chat_xpath)
    new_chat_entry.click()

In [50]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
time.sleep(1)
driver.get("https://twllm.com/")

b1_xpath = '/html/body/div[2]/div/div/div/div/button'
b1_input = driver.find_element(By.XPATH, b1_xpath)
b1_input.click()

target_map = {
    'price_synonums': {'高(形容物品價格)': [], '還好(形容物品價格)': [], '低(形容物品價格)': []},
    'size_synonyms': {'大(形容電鍋容量)': [], '剛剛好(形容電鍋容量)': [], '小(形容電鍋容量)': []},
    'color_synonyms': {'白色': [], '紅色': [], '黑色': []},

}

for synonym_key, data in target_map.items():
    for key, val in data.items():
        scraperLoop(key, target_map[synonym_key], driver, 10)

driver.quit()

In [53]:

actual_map = {}
for sy_key, sy_map in target_map.items():
    if sy_key == '': continue
    actual_map[sy_key] = {}
    for key, val in sy_map.items():
        for subval in val:
            if subval == '': continue
            actual_map[sy_key][subval] = key


In [57]:
with open('feature_map.plk', 'rb') as f:
    old_map = pickle.load(f)


In [ ]:
old_map.update(actual_map)
with open('feature_map.plk', 'wb') as f:
    pickle.dump(old_map, f)